In [1]:
import os

import pandas as pd

In [2]:
# 自訂函式
def clear_col_str(col):
    col = col.str.replace("※", "")
    col = col.str.replace("　", "")
    col = col.str.replace(" ", "")
    col = col.str.strip()
    
    return col

In [3]:
# 讀取excel檔案中的所有分頁
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data"
file = "population.xls"
file_path = os.path.join(folder, file)

xls = pd.read_excel(file_path, sheet_name=None)

In [4]:
# 定義欄位名和建立主要dataframe
columns = ["district", "household_count", "population", "male", "female"]
new_col = ['city', 'district', 'household_count', 'population', 'male', 'female']

df_main = pd.DataFrame(columns=new_col)

# 取得所有分頁名，並且從新北市開始（忽略總計）
sheet_list = list(xls.keys())
start = sheet_list.index("新北市")

In [5]:
# 根據分頁開始逐縣市迴圈
for n in range(start, len(sheet_list)):
    city_name = sheet_list[n]
    df = xls.get(city_name, None)

    # 重定義欄位名
    df.columns = columns

    # 去除非資料的row
    df.drop(df.index[[0, 1, -1]], inplace=True)

    # 清理地區名中的空格和特殊字串
    df["district"] = clear_col_str(df["district"])

    # 將人口數相關欄位的資料都轉換成int型別
    col_list = list(df.columns)

    for n in range(1, len(col_list)):
        df[col_list[n]] = df[col_list[n]].astype(int)

    # 增加一欄"city"
    df["city"] = df["district"].iloc[0]

    # 重新排列欄位順序
    df = df[new_col]

    # 與主要dataframe結合
    df_main = pd.concat([df_main, df])

In [ ]:
# 每個縣市都處理完後，將主要dataframe存檔
main_file = "population_data.csv"
main_path = os.path.join(folder, main_file)

df_main.to_csv(main_path, index=False, encoding="utf-8")

In [7]:
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

df_main.to_sql(name="raw_population", con=engine, if_exists="replace", index=False)

print("資料已輸入資料庫！")

資料已輸入資料庫！
